# OLD

In [ ]:
import numpy as np
import pandas as pd

import findspark
findspark.init()

import pyspark
import random

from pyspark import SparkContext
from pyspark.sql import SQLContext

sc = pyspark.SparkContext(appName="Pi")

#sc = SparkContext()
sqlContext = SQLContext(sc)

In [ ]:
from pyspark.sql import SparkSession
sparkSession = SparkSession.builder.appName("example-pyspark-read-and-write").getOrCreate()

#SparkConf().set("spark.sql.execution.arrow.enabled", "true")

df = sparkSession.read.parquet('hdfs://172.30.17.145:8020/sla_sql_data/1580137124017/part-00000-b2678865-5b2e-43a7-8428-117b6f435dc5-c000.snappy.parquet')
#df.show()
#pf = df.toPandas()

#df = sqlContext.read.parquet("/home/jovyan/work/notebook/flat/000000_1")

In [ ]:
#pf = df.limit(1000000).toPandas()
#pf=df.filter(df["CSERVICE"]=='PEPPOL').select('*').limit(1000000).dropDuplicates().toPandas()
#pf=df.filter(df["CSERVICE"]=='PEPPOL').select('*').limit(1000000).toPandas()

In [ ]:
#pf1 = df.filter(df["CSERVICE"]=='PEPPOL').select('*').limit(1000000).dropDuplicates()
pf = pf1.toPandas()

In [ ]:
pf = pf.sort_values('CGLOBALMESSAGEID', ascending=True)

In [ ]:
pf["CSTARTTIME"] = pd.to_numeric(pf["CSTARTTIME"])
#set(pf['CSTARTTIME'].dt.date)

In [ ]:
#pd.to_datetime(df['CENDTIME']).apply(lambda x: x.date())
import datetime
readable = datetime.datetime.fromtimestamp(1580983470).isoformat()
print(readable)

ab = pf.iloc[121]['CENDTIME']

import datetime
date = datetime.datetime.fromtimestamp(float(ab) / 1e3)

date

In [ ]:
import pandas_profiling
import numpy as np
pandas_profiling.ProfileReport(pf)

In [ ]:
import numpy as np   

def get_categories(column):
    return np.array(df.select(column).dropDuplicates().dropna().collect())

services = get_categories('CSERVICE')
receivers = get_categories('CRECEIVERENDPOINTID')
senders = get_categories('CSENDERENDPOINTID')
receiverprotocols = get_categories('CRECEIVERPROTOCOL')
senderprotocols = get_categories('CSENDERPROTOCOL')
status = get_categories('CSTATUS')

In [ ]:
services

# Hive

In [ ]:
from sqlalchemy import create_engine
from pyhive import hive

from sqlalchemy import *
from sqlalchemy.engine import create_engine
from sqlalchemy.schema import *

#engine = create_engine('hive://clspromon-aio01:10000/default')
engine = create_engine('hive://172.30.17.145:10000/default')

In [ ]:
inspector = inspect(engine)

# Get table information
#print(inspector.get_table_names())

# Get column information
row=inspector.get_columns('tables_stage1_p_meta_complete')

col = [] 
for _row in row:
    #print(_row['name'])
    col.append(_row['name'])

#col

import pandas as pd

row =  engine.execute("select * from  tables_stage1_p_meta_complete")
meta_p = pd.DataFrame(row)

meta_p.columns = col
#pyspark.createDataFrame(meta_p).collect()  

meta_p.set_index('metricid', inplace=True)
#meta_p.set_index('metricid')

meta_p['metricid'] = meta_p.index

#meta_p.instance.fillna(value=pd.np.nan, inplace=True)
meta_p['instance'].replace(pd.np.nan, "", inplace=True)


In [ ]:
#meta_p.loc[-415304450].__name__
#meta_p.iloc[1].service
#meta_p
#meta_p.loc[meta_p.index[0]].__name__
#meta_p.index[1]
#meta_p.iloc[1]
col

In [ ]:
#meta_p.info()
dfm = sqlContext.createDataFrame(meta_p)

In [ ]:
import numpy as np   

#objects = dfm.select('__name__','instance','product', 'metricid', 'schemaid').dropDuplicates()
services = np.array(dfm.select('service').dropDuplicates().dropna().collect())
names = np.array(dfm.select('__name__').dropDuplicates().dropna().collect())
instances = np.array(dfm.select('instance').dropDuplicates().dropna().collect())
products = np.array(dfm.select('product').dropDuplicates().dropna().collect())
schemaids = np.array(dfm.select('schemaid').dropDuplicates().dropna().collect())
metricid = np.array(dfm.select('metricid').dropDuplicates().dropna().collect())

In [ ]:
#roles = np.array(dfm.select('role').dropDuplicates().dropna().collect())
#roles
schemaids.size

In [ ]:
#print(str(metricid[1][0]))
#names.size
#dfm.select('__name__').dropDuplicates().dropna().filter("__name__ like '%task%'").collect()

#.filter(dfm.__name__.contains('%b%')).show()
#dfm.filter(dfm["__name__"]=='bis_inbox_tasks_running').select("*").collect()
#dfm.filter(dfm["instance"]=='clspromsg-ts01:13000').select("__name__").collect()
#dfm.filter(dfm["instance"]=='clspromsg-ids01:13000').select("__name__").collect()
#dfm.filter(dfm["__name__"]=='bis_jms_message_count').select("instance",'metricid').collect()

#la = ['__name__','instance','metricid','schemaid','job','alertname','alertstate','severity' ]
#dfm.filter(dfm["__name__"]=='bis_jms_message_count').filter(dfm["queue"]=='DtFTPComponent').filter(dfm["instance"]=='clspromsg-ts01:13000').select(la).collect()

#alertes = dfm.filter(dfm["__name__"]=='ALERTS').select(la).limit(200000).toPandas()
#alertes.schemaid.size
#instances

In [ ]:
la = ['__name__','instance','metricid','schemaid','job','alertname','alertstate','severity' ]

alerts = dfm.filter(dfm["__name__"]=='ALERTS').select(la).limit(200000).toPandas()
alerts.schemaid.size

In [ ]:
df.filter(df["hashvalue"]==id).select('value').distinct().count()

#pf.describe()

In [ ]:
for _row in metricid:
    id = str(_row[0])
    print(id,df.filter(df["hashvalue"]==id).select('value').distinct().count())

id = '-1483665529'
#id = str(meta_p.index[0])

In [ ]:
#metricid[0]

id = 1797174388 
#id = '-415304450'

#id = -1756241224 #1
#id = 1843265645 #24497
#id = 852430459 #1
#id = -426467024 #22515
#id = 6557551 #25097
#id = -1466543823 #26614
#id = 612749002 #2
#id = -547512297 #24905
#id = -415304450 #4920
#id = 857353913 #1
#id = 500298778 #1
#id = -349732210 #2
#id = 1786297729 #26414

#pf=df.filter(df["value"]>'0').filter(df["hashvalue"]=='2012513398').select('date','value').limit(200000).toPandas()
#pf=df.filter(df["value"]>'0').filter(df["hashvalue"]==str(metricid[0])).select('date','value').limit(200000).toPandas()
#pf=df.filter(df["hashvalue"]==str(metricid[1000][0])).select('date','value').limit(200000).toPandas()


In [ ]:
def get_pf(id):
    pf=df.filter(df["hashvalue"]==str(id)).select('date','value').limit(200000).toPandas()
    pf = pf.sort_values('date', ascending=True)
    pf["value"] = pd.to_numeric(pf["value"])
    return pf

def get_plot_title(id) :
    title = str(id) + " = " + meta_p.loc[id].__name__ + " -> " + meta_p.loc[id].instance
    return title

id = alerts.metricid[3]
id=612749002

pf = get_pf(id)
#alertes.metricid.size

In [ ]:
id=612749002
#val = df.filter(df["hashvalue"]==str(id)).select('value')

#val.distinct().collect()
#val.distinct().count()

#val.groupBy('value').count().show()

df.filter(df["hashvalue"]==str(id)).filter(df["value"]=='8').select("*").show()



In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plot_title = get_plot_title(id)

pf.plot(x='date', y='value', title=plot_title)

In [ ]:
#df.head()
pf = pf.sort_values('date', ascending=True)
pf.count()
#pf = pf.astype('float')
pf["value"] = pd.to_numeric(pf["value"])
print(type(pf['value'][0]))

In [ ]:
from pyspark.sql import functions as F
#df.select("hashvalue", F.when(df.hashvalue == 266655713, 1).otherwise(0)).show()
#df.select("hashvalue", F.when(df.hashvalue == 266655713, 1).otherwise(0)).show()
#df.select(df.hashvalue.between(266655713,266655713)).show()
#df.filter(df["hashvalue"]==str(metricid[1][0])).select('date','hashvalue').rdd.flatMap(list).collect()
df.filter(df["hashvalue"]==str(metricid[1][0])).select("*").limit(2).toPandas()

In [ ]:
print(metricid[0][0])
len(metricid)

for row in products:
    print(row[0])

for id in metricid:
    pf=df.filter(df["value"]>'0').filter(df["hashvalue"]==id[0]).select('date','value').limit(200000).toPandas()
    pf.describe()
    

In [ ]:
services.count()
products.show()

for row in services.rdd.collect():
    print(row.service)
    
 
#np.array(services.select("service").collect())
services

In [ ]:
hashes = [] 
for _row in row:
    print(_row['hashvalue'])
    hashes.append(_row['hashvalue'])

#col

In [ ]:
#names.select("*").collect()
#hashes
meta_p